# Travel Deals ✈️

Airlines frequently market their flights by presenting a lineup of departure airports. Travelers must navigate through this list to locate their local airport and check out the latest promotions. Artificial intelligence can provide valuable assistance in this regard. Our journey commences with up-to-the-minute deals from Amadeus. During the journey, we leverage some unique methods to generate test data. In the end, we employ Semantic Kernel and Azure OpenAI to craft personalized emails that we can send directly to our target customers, featuring deals from their preferred airport.

Instructions for getting started with this Ultralight Backpackers AI trek are on the [GitHub page's README](../README.md). 

# Ultralight gear list for AI-driven travel merchandising🧗 

- 1 JSON fromatted list of flight deals from your home airport.  Register as a developer on the [Amadeus API developer portal](https://developers.amadeus.com/).
- 1 [Semantic Plugin TestData](./plugins-sk/TripPlans/TestData/skprompt.txt) to create test data.
- 1 [Semantic Plugin EmailContent](./plugins-sk/TripPlans/EmailContent/skprompt.txt) acts as a travel marketing writer.



> [!IMPORTANT]
> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

# Gather gear for our trek 🔦🥾⛺

When running the following cell, if asked to "select your kernel" (note this will be referring to the Jupyter notebook's kernel and not Semantic Kernel) then choose `.NET Interactive` from the available menu options.

In [ ]:
// Load some helper functions, e.g. to load values from settings.json
#!import ../config/Settings.cs 
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: System.Net.Http.Json, *-*"



# Pack gear into the kernel 🚙
We only pack what we need.  Semantic Kernel comes with a handy built-in HTTP plugin.  We're also importing two semantic functions.  One to create data for testing, and one that transforms real time pricing offers into an email.

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;
using System;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Threading.Tasks;

var builder = new KernelBuilder();
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, tomorrowioApiKey, amadeusClientID, amadeusSecret) = Settings.LoadFromFile();
var token = await Settings.ConnectOAuth(amadeusClientID, amadeusSecret);
Console.WriteLine($"Using 🧱 Model: {model}");

builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
IKernel kernel = builder.Build();

var client = new HttpClient() { 
    BaseAddress = new Uri("https://test.api.amadeus.com"), // <-LLMs need real ⏱️time data help
    DefaultRequestHeaders =
    {
        Authorization =
            new AuthenticationHeaderValue("Bearer", token)
    }
};
var http = kernel.ImportFunctions(new HttpPlugin(client), "http");
var trip = kernel.ImportSemanticFunctionsFromDirectory("plugins-sk", "TripPlans");


#  Resupply: use a🔌plugin to create test 📄data

In [ ]:
var testData = trip["TestData"].InvokeAsync(kernel);
var testDataStr = testData.Result.GetValue<string>();
var testDataArr = testDataStr.Split(',');
Console.WriteLine("🧪test data " + testDataStr);

# Trail magic: 🔌plugin pipeline 

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Plugins.Core;

var trailMagic = new List<string>();
foreach(var airport in testDataArr) {
    var context = new ContextVariables(); 
    context["uri"] = $"v1/shopping/flight-destinations?origin={airport.Trim()}&oneWay=false&nonStop=false";
    
    try {
        Console.WriteLine(context["uri"]);
        // REST ---> JSON ---> Email formatting plugin ⬇️
        var results = await kernel.RunAsync(context, http["get"],trip["EmailContent"]);
        Console.WriteLine(results.GetValue<string>());
        trailMagic.Add(results.GetValue<string>());
    } catch (Exception e) {
        Console.WriteLine("no prices for this one");    
    }
    
}




# 🚶Hiker 🗒️log, show what we saw and did

In [ ]:
using Microsoft.AspNetCore.Html;
using Markdig;

var myPipeline = new MarkdownPipelineBuilder().UseAdvancedExtensions().Build();

var myHTMLContent = new HtmlContentBuilder();

foreach (var email in trailMagic) {
    var myResult2HTML = Markdown.ToHtml("# ---------\n" + email, myPipeline);
    myHTMLContent.AppendHtml(myResult2HTML);
}
myHTMLContent

